# Introduction
Models are trained in google colab with batch_size of 200-700. For online prediction, we only need batch_size of 1.
Here we recombined models in 

### Get source folder and append to sys directory

In [1]:
from __future__ import print_function
import os
import sys
PROJ_ROOT = os.path.join(os.pardir)
print(os.path.abspath(PROJ_ROOT))
src_dir = os.path.join(PROJ_ROOT, "src")
sys.path.append(src_dir)
# Data path example
#pump_data_path = os.path.join(PROJ_ROOT,
#                              "data",
#                              "raw",
#                              "pumps_train_values.csv")

/mnt/4ba37af6-51fd-47bc-8321-8c500c229114/study/School/KHOA LUAN TOT NGHIEP/runnable_program


### Imports
Import libraries and write settings here.

In [2]:
# Data manipulation
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import sklearn
import random

# Options for pandas
pd.options.display.max_columns = 50
pd.options.display.max_rows = 30
# Display all cell outputs
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'
from IPython import get_ipython
ipython = get_ipython()
# Visualizations
import matplotlib.pyplot as plt

In [8]:
import pandas as pd
import os
# import tensorflow as tf
import googleapiclient.discovery
from google.api_core.client_options import ClientOptions
import feedparser
import numpy as np
from io import StringIO
from google.cloud import storage
from features import extract_features
from data import create_load_transform_processed_data

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/mnt/4ba37af6-51fd-47bc-8321-8c500c229114/study/School/KHOA LUAN TOT NGHIEP/runnable_program/flask_app/flask-app-test-317210-bdec872c665d.json"
PROJECT = "flask-app-test-317210" # change for your GCP project
REGION = "us-central1" # change for your GCP region (where your model is hosted)
MODEL = ['hour_1', 'hour_2', 'hour_3', 'hour_4', 'hour_5']

def predict_json(project, region, model, instances, version=None):
    """Send json data to a deployed model for prediction.

    Args:
        project (str): project where the Cloud ML Engine Model is deployed.
        model (str): model name.
        instances ([Mapping[str: Any]]): Keys should be the names of Tensors
            your deployed model expects as inputs. Values should be datatypes
            convertible to Tensors, or (potentially nested) lists of datatypes
            convertible to Tensors.
        version (str): version of the model to target.
    Returns:
        Mapping[str: any]: dictionary of prediction results defined by the 
            model.
    """
    # Create the ML Engine service object
    prefix = "{}-ml".format(region) if region else "ml"
    api_endpoint = "https://{}.googleapis.com".format(prefix)
    client_options = ClientOptions(api_endpoint=api_endpoint)

    # Setup model path
    model_path = "projects/{}/models/{}".format(project, model)
    if version is not None:
        model_path += "/versions/{}".format(version)

    # Create ML engine resource endpoint and input data
    ml_resource = googleapiclient.discovery.build(
        "ml", "v1", cache_discovery=False, client_options=client_options).projects()
    # turn input into list (ML Engine wants JSON)
    instances_list = instances.tolist()

    input_data_json = {"signature_name": "serving_default",
                       "instances": instances_list}

    request = ml_resource.predict(name=model_path, body=input_data_json)
    response = request.execute()

    # # ALT: Create model api
    # model_api = api_endpoint + model_path + ":predict"
    # headers = {"Authorization": "Bearer " + token}
    # response = requests.post(model_api, json=input_data_json, headers=headers)

    if "error" in response:
        raise RuntimeError(response["error"])

    scaler = extract_features.load_scaler()
    reverse_scaled_prediction = scaler.inverse_transform(response['predictions'])

    return reverse_scaled_prediction

In [13]:
# Data path example
processed_data_path = os.path.join(PROJ_ROOT,
                             "data",
                             "processed",
                             "past_data.csv")
data = pd.read_csv(processed_data_path)
data

,site_id,time,AQI_h,AQI_h_label,AQI_h_I,Continous length
0,49,2021-07-01 02:00:00,31.0,Good,1,0
1,49,2021-07-01 03:00:00,31.0,Good,1,0
2,49,2021-07-01 04:00:00,31.0,Good,1,0
3,49,2021-07-01 05:00:00,24.0,Good,1,0
4,49,2021-07-01 06:00:00,28.0,Good,1,0
...,...,...,...,...,...,...
364,49,2021-07-16 22:00:00,57.0,Moderate,2,0
365,49,2021-07-16 23:00:00,61.0,Moderate,2,0
366,49,2021-07-17 00:00:00,59.0,Moderate,2,0
367,49,2021-07-17 01:00:00,54.0,Moderate,2,0


In [21]:
import random
random_number = np.random.uniform(-15, 15, data.shape[0])
data['AQI_h'] = data['AQI_h'].add(random_number, axis=0).round()
data['AQI_h'] = data['AQI_h'].astype(float)
data.to_csv(processed_data_path)

In [4]:
def get_data_from_bucket_as_dataframe(filename="past_data.csv"):
    """Read a blob"""
    bucket_name = "deep_learning_model_bucket"

    storage_client = storage.Client()

    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(filename)
    if blob.exists() == False:
        return None

    return_data = blob.download_as_text()
    return_data = StringIO(return_data)
    df = pd.read_csv(return_data, sep=",", header=0, index_col=False)
    return df
data_df = get_data_from_bucket_as_dataframe()
data_df = data_df.tail(25)
data_df = data_df.astype({'time': 'datetime64[ns]', 'AQI_h': 'float'})
data_df.set_index(['site_id', 'time'], inplace=True)
data_df

AQI_h AQI_h_label  AQI_h_I  Continous length
site_id time                                                             
49      2021-07-16 01:00:00   22.0        Good        1                 0
        2021-07-16 02:00:00   30.0        Good        1                 0
        2021-07-16 03:00:00   50.0        Good        1                 0
        2021-07-16 04:00:00   46.0        Good        1                 0
        2021-07-16 05:00:00   58.0    Moderate        2                 0
        2021-07-16 06:00:00   75.0    Moderate        2                 0
        2021-07-16 07:00:00   58.0    Moderate        2                 0
        2021-07-16 08:00:00   46.0        Good        1                 0
        2021-07-16 09:00:00   35.0        Good        1                 0
        2021-07-16 10:00:00   34.0        Good        1                 0
        2021-07-16 11:00:00   32.0        Good        1                 0
        2021-07-16 12:00:00   28.0        Good        1                 0
        2021-07-16 13:00:00   27.0        Good        1                 0
        2021-07-16 14:00:00   26.0        Good        1                 0
        2021-07-16 15:00:00   25.0        Good        1                 0
        2021-07-16 16:00:00   23.0        Good        1                 0
        2021-07-16 17:00:00   22.0        Good        1                 0
        2021-07-16 18:00:00   29.0        Good        1                 0
        2021-07-16 19:00:00   44.0        Good        1                 0
        2021-07-16 20:00:00   53.0    Moderate        2                 0
        2021-07-16 21:00:00   52.0    Moderate        2                 0
        2021-07-16 22:00:00   57.0    Moderate        2                 0
        2021-07-16 23:00:00   61.0    Moderate        2                 0
        2021-07-17 00:00:00   59.0    Moderate        2                 0
        2021-07-17 01:00:00   54.0    Moderate        2                 0

In [5]:
def create_input_for_model(df, timesteps=[1], target_hour=[1], test_output=False, dev_output=False, output_path=None, PROJ_ROOT=os.pardir):
    """From interim dataframe:
        + add features
        + split into chunks according to timesteps
        + compressed and saved to output_path
        + estimate number of created dataset = timesteps * target_hour
    Parameters
    ----------
    df : pandas.DataFrame
        Contains interim data.
    timesteps : list of integer
        Each timestep represent 1 dataset
    target_hour : list of integer
        the label for each timesteps
    output_path : string
        Destination directory the dataset will be created
    """
    if output_path == None:
        output_path == os.path.join(PROJ_ROOT,
                                    "data",
                                    "model_input")
    for timesteps in timesteps:
        for target_hour in target_hour:
            # Create train, dev, test data
            train_df = extract_features.create_and_save_scale_data(df, output_path=output_path).copy()
            train_df = extract_features.add_features(train_df).copy()
            if test_output is not False:
                train_df, test_df = extract_features.generate_train_test_set_by_time(
                    train_df)
                test, y_test, multiclass_y_test = extract_features.data_preprocessing(
                    test_df, target_hour, timesteps=timesteps)
                create_load_transform_processed_data.reshape_array_and_save_to_path(
                    test, y_test, path=output_path, timesteps=timesteps, target_hour=target_hour, data_type="test")
            if dev_output is not False:
                train_df, dev_df = extract_features.generate_train_test_set_by_time(
                    train_df)
                dev, y_dev, multiclass_y_dev = extract_features.data_preprocessing(
                    dev_df, target_hour, timesteps=timesteps)
                create_load_transform_processed_data.reshape_array_and_save_to_path(
                    dev, y_dev, path=output_path, timesteps=timesteps, target_hour=target_hour, data_type="dev")

            train, y_train, multiclass_y = extract_features.data_preprocessing(
                train_df, target_hour, timesteps=timesteps)

            # Save data to file
            if output_path is not None:
                create_load_transform_processed_data.reshape_array_and_save_to_path(
                    train, y_train, path=output_path, timesteps=timesteps, target_hour=target_hour, data_type="train")
    train = train.astype('float32')
    y_train = y_train.astype('float32')
    print("Input have been created")
    return train, y_train
data_df = data_df.tail(48)
predict_data, label = create_input_for_model(data_df, timesteps=[5], target_hour=[1])
predict_data

0
5
0 days 05:00:00
Feature shape:  (13, 5, 34)
Label shape:  (13,)
Input have been created


array([[[ 0.05660377,  0.        ,  0.        , ..., -0.8490566 ,
         -0.8490566 , -0.8490566 ],
        [-0.09433962,  0.        ,  0.        , ..., -0.5471698 ,
         -0.5471698 , -0.5471698 ],
        [ 0.35849056,  1.        ,  0.        , ..., -0.43396226,
         -0.43396226, -0.43396226],
        [ 1.        ,  1.        ,  0.        , ..., -0.2754717 ,
         -0.2754717 , -0.2754717 ],
        [ 0.35849056,  1.        ,  0.        , ..., -0.06289309,
         -0.06289309, -0.06289309]],

       [[-0.09433962,  0.        ,  0.        , ..., -0.5471698 ,
         -0.5471698 , -0.5471698 ],
        [ 0.35849056,  1.        ,  0.        , ..., -0.43396226,
         -0.43396226, -0.43396226],
        [ 1.        ,  1.        ,  0.        , ..., -0.2754717 ,
         -0.2754717 , -0.2754717 ],
        [ 0.35849056,  1.        ,  0.        , ..., -0.06289309,
         -0.06289309, -0.06289309],
        [-0.09433962,  0.        ,  0.        , ..., -0.00269542,
         -0.00

In [6]:
tmp_predict_data = predict_data[-1].copy()
tmp_predict_data = np.reshape(tmp_predict_data, (1, predict_data.shape[1], predict_data.shape[2]))
tmp_predict_data.shape

(1, 5, 34)

In [20]:
past_prediction = get_data_from_bucket_as_dataframe(
        filename="past_data.csv")
past_prediction['AQI_h']

0      31.0
1      31.0
2      31.0
3      24.0
4      28.0
       ... 
364    57.0
365    61.0
366    59.0
367    54.0
368    40.0
Name: AQI_h, Length: 369, dtype: float64

In [9]:
all_result = np.array([])
for model in range(0,5):
    preds = predict_json(project=PROJECT,
                                region=REGION,
                                model=MODEL[model],
                                instances=tmp_predict_data)
    all_result = np.append(all_result, preds)
all_result

array([41.44885257, 34.36666909, 34.52255065, 33.09760285, 31.69491367])

In [ ]:
# Now let's load and scale predicted data


# Analysis/Modeling

# Results
Show graphs and stats here

# Conclusions and Next Steps
Summarize findings here